In [ ]:
import requests
import xmltodict
import json
from pprint import pprint
from tqdm.notebook import tqdm
import pandas as pd

In [ ]:
url = 'http://apis.data.go.kr/1192000/select0090List/getselect0090List'

In [ ]:
api_key = 'PBJct2NnY43P0PBlqdr3cVao2RJ5bJ0BQOgV9PIOdQFjCCG259NS9FMREH8bJILFOMoxa35pQsOSmLw7Qawprg=='

In [ ]:
query = {
    'serviceKey': api_key, 
    'numOfRows': 100, 
    'pageNo': 1, 
    'type': 'json', 
    'baseDt': '202401'
}

In [ ]:
res = requests.get(url = url, params = query)

In [ ]:
res.status_code

In [ ]:
res.headers['content-type']

In [ ]:
res.text
type(res.text)

In [ ]:
# json -> dict
dat = json.loads(res.text)
type(dat)

In [ ]:
pprint(dat)

In [ ]:
totalCount = int(dat['responseJson']['header']['totalCount'])
totalCount

In [ ]:
rows = 100
pages = int(totalCount / rows) + 1
pages

In [ ]:
df = pd.DataFrame(data = dat['responseJson']['body']['item'])
df.shape

In [ ]:
def mprcTotalCount(ymonth, rows = 100):
    url = 'http://apis.data.go.kr/1192000/select0090List/getselect0090List'
    query = {
        'serviceKey': 'PBJct2NnY43P0PBlqdr3cVao2RJ5bJ0BQOgV9PIOdQFjCCG259NS9FMREH8bJILFOMoxa35pQsOSmLw7Qawprg==', 
        'numOfRows': rows, 
        'pageNo': 1, 
        'type': 'json', 
        'baseDt': ymonth
    }
    res = requests.get(url = url, params = query)
    dat = json.loads(res.text)
    totalCount = int(dat['responseJson']['header']['totalCount'])
    pages = int(totalCount / rows) + 1
    return totalCount, pages

In [ ]:
totalCount, pages = mprcTotalCount(ymonth = '202401')
print(totalCount)
print(pages)

In [ ]:
def mprcDataFrame(ymonth, rows = 100, page = 1):
    url = 'http://apis.data.go.kr/1192000/select0090List/getselect0090List'
    query = {
        'serviceKey': 'PBJct2NnY43P0PBlqdr3cVao2RJ5bJ0BQOgV9PIOdQFjCCG259NS9FMREH8bJILFOMoxa35pQsOSmLw7Qawprg==', 
        'numOfRows': rows, 
        'pageNo': page, 
        'type': 'json', 
        'baseDt': ymonth
    }
    res = requests.get(url = url, params = query)
    dat = json.loads(res.text)
    df = pd.DataFrame(data = dat['responseJson']['body']['item'])
    return df

In [ ]:
df = mprcDataFrame(ymonth = '202401')
df.shape

In [ ]:
dfs = pd.DataFrame()
for page in tqdm(range(pages)):
    page += 1
    df = mprcDataFrame(ymonth = '202401', page = page)
    dfs = pd.concat(objs = [dfs, df])

In [ ]:
dfs.shape

In [ ]:
def mprcDataset(ymonth):
    totalCount, pages = mprcTotalCount(ymonth = ymonth)
    dfs = pd.DataFrame()
    for page in tqdm(range(pages)):
        page += 1
        df = mprcDataFrame(ymonth = ymonth, page = page)
        dfs = pd.concat(objs = [dfs, df])
    return dfs

In [ ]:
df = mprcDataset(ymonth = '202401')
df.shape

In [ ]:
ymonths = range(202401, 202413)
dfs = pd.DataFrame()
for ymonth in tqdm(ymonths):
    df = mprcDataset(ymonth = ymonth)
    dfs = pd.concat(objs = [dfs, df])

In [ ]:
dfs.shape

In [ ]:
dfs

In [ ]:
dfs = dfs.reset_index(drop = True)

In [ ]:
import os

In [ ]:
os.getcwd()

In [ ]:
os.chdir(path = 'C:\\Users\\user\\Documents\\project\\data')
os.getcwd()

In [ ]:
%time dfs.to_excel('MPRC_2024.xlsx', index = False)

In [ ]:
%time dfs.to_csv('MPRC_2024.csv', index = False, encoding = 'CP949')

In [ ]:
os.listdir()

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
os.getcwd()

'C:\\Users\\user\\Documents\\project\\code'

In [3]:
# 상대 경로: '.'(현재 폴더), '..'(부모 폴더)
os.chdir(path = '../data')
os.getcwd()

'C:\\Users\\user\\Documents\\project\\data'

In [4]:
os.listdir()

['MPRC_2024.csv', 'MPRC_2024.xlsx']

In [5]:
os.chdir(path = '../code')
os.listdir()

['.ipynb_checkpoints',
 'Day1.ipynb',
 'Day2.ipynb',
 'Day3.ipynb',
 'Day4.ipynb',
 'first_code.ipynb',
 'myfuncs.py',
 '__pycache__']

In [6]:
os.chdir(path = '../data')
os.getcwd()

'C:\\Users\\user\\Documents\\project\\data'

In [7]:
os.listdir()

['MPRC_2024.csv', 'MPRC_2024.xlsx']

In [8]:
%time df = pd.read_excel('MPRC_2024.xlsx')

CPU times: total: 8.5 s
Wall time: 13.5 s


In [11]:
%time df = pd.read_csv('MPRC_2024.csv', encoding = 'CP949')

CPU times: total: 46.9 ms
Wall time: 150 ms


In [13]:
df.head(n = 3)

,stdYymm,nationCode,mprcExipitmCode,imxprtSeCode,nationNm,ecoblNm,mprcExipitmNm,imxprtSeNm,imxprtWt,imxprtDollarAmount
0,202401,AE,106199000,E,아랍에미리트연합,걸프협력회의,기타 포유동물,수출,17.5,455
1,202401,AE,301998000,E,아랍에미리트연합,걸프협력회의,넙치(활어),수출,798.2,19735
2,202401,AE,302240000,E,아랍에미리트연합,걸프협력회의,터벗(신선 또는 냉장)(프세타 맥시마),수출,304.0,11425


In [14]:
df.tail()

,stdYymm,nationCode,mprcExipitmCode,imxprtSeCode,nationNm,ecoblNm,mprcExipitmNm,imxprtSeNm,imxprtWt,imxprtDollarAmount
32375,202412,VU,304879000,I,바누아트,NaN,다랑어 기타(냉동한 피레트),수입,819.400,7232
32376,202412,VU,307432090,I,바누아트,NaN,기타,수입,6499.350,29897
32377,202412,ZA,106199000,I,남아프리카,NaN,기타 포유동물,수입,6.000,1790
32378,202412,ZA,303821000,I,남아프리카,NaN,가오리(냉동),수입,975.000,3022
32379,202412,ZA,303892000,I,남아프리카,NaN,갈치(냉동),수입,301397.525,625530


In [20]:
# 재현 가능한 결과를 얻기 위해 초기값을 고정해줍니다.
df.sample(random_state = 1)

,stdYymm,nationCode,mprcExipitmCode,imxprtSeCode,nationNm,ecoblNm,mprcExipitmNm,imxprtSeNm,imxprtWt,imxprtDollarAmount
9486,202404,VU,303550000,E,바누아트,NaN,전갱이(냉동)(트라커러스종),수출,15200.0,19250


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32380 entries, 0 to 32379
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   stdYymm             32380 non-null  int64  
 1   nationCode          32371 non-null  object 
 2   mprcExipitmCode     32380 non-null  int64  
 3   imxprtSeCode        32380 non-null  object 
 4   nationNm            32380 non-null  object 
 5   ecoblNm             27466 non-null  object 
 6   mprcExipitmNm       32380 non-null  object 
 7   imxprtSeNm          32380 non-null  object 
 8   imxprtWt            32380 non-null  float64
 9   imxprtDollarAmount  32380 non-null  int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 2.5+ MB


In [22]:
df.columns

Index(['stdYymm', 'nationCode', 'mprcExipitmCode', 'imxprtSeCode', 'nationNm',
       'ecoblNm', 'mprcExipitmNm', 'imxprtSeNm', 'imxprtWt',
       'imxprtDollarAmount'],
      dtype='object')

In [23]:
df.columns = ['기준년월', '국가코드', '수산물코드', '수출입코드', '국가명',
              '경제블럭명', '수산물명', '수출입명', '중량', '달러']

In [24]:
df.head()

,기준년월,국가코드,수산물코드,수출입코드,국가명,경제블럭명,수산물명,수출입명,중량,달러
0,202401,AE,106199000,E,아랍에미리트연합,걸프협력회의,기타 포유동물,수출,17.5,455
1,202401,AE,301998000,E,아랍에미리트연합,걸프협력회의,넙치(활어),수출,798.2,19735
2,202401,AE,302240000,E,아랍에미리트연합,걸프협력회의,터벗(신선 또는 냉장)(프세타 맥시마),수출,304.0,11425
3,202401,AE,303510000,E,아랍에미리트연합,걸프협력회의,"청어(클루페아 하렌구스,클루페아 팔라시)",수출,20.0,638
4,202401,AE,303540000,E,아랍에미리트연합,걸프협력회의,"고등어(스콤버 스콤브루스,스콤버 오스트랄라시쿠스,스콤버 자포니쿠스)",수출,170.0,1472


In [26]:
# df.drop(columns = ['국가코드', '수산물코드', '수출입코드', '경제블럭명'], inplace = True)
df = df.drop(columns = ['국가코드', '수산물코드', '수출입코드', '경제블럭명'])

In [27]:
df.shape

(32380, 6)

In [28]:
df.dtypes

기준년월      int64
국가명      object
수산물명     object
수출입명     object
중량      float64
달러        int64
dtype: object

In [31]:
df['기준년월'] = df['기준년월'].astype(str)

In [32]:
df.dtypes

기준년월     object
국가명      object
수산물명     object
수출입명     object
중량      float64
달러        int64
dtype: object

In [33]:
df.head()

,기준년월,국가명,수산물명,수출입명,중량,달러
0,202401,아랍에미리트연합,기타 포유동물,수출,17.5,455
1,202401,아랍에미리트연합,넙치(활어),수출,798.2,19735
2,202401,아랍에미리트연합,터벗(신선 또는 냉장)(프세타 맥시마),수출,304.0,11425
3,202401,아랍에미리트연합,"청어(클루페아 하렌구스,클루페아 팔라시)",수출,20.0,638
4,202401,아랍에미리트연합,"고등어(스콤버 스콤브루스,스콤버 오스트랄라시쿠스,스콤버 자포니쿠스)",수출,170.0,1472


In [45]:
df['수산물명'].unique()

array(['기타 포유동물', '넙치(활어)', '터벗(신선 또는 냉장)(프세타 맥시마)',
       '청어(클루페아 하렌구스,클루페아 팔라시)', '고등어(스콤버 스콤브루스,스콤버 오스트랄라시쿠스,스콤버 자포니쿠스)',
       '삼치[스콤버로모러스(Scomberomorus)종]', '명태(냉동)(테라그라 찰코그라마)', '복어(냉동)',
       '임연수어(냉동)', '가자미(냉동한 피레트)', '어란(명태/염장,염수장)',
       '멸치[엔그라울리스(Engraulis)종]', '조기(건조/염장불문/훈제제외)',
       '굴치패 기타(산 것 신선 또는 냉장한 것)', '냉동한 것', '기타', '문어', '낙지', '바지락',
       '전복(산 것 신선 또는 냉장한 것)', '냉동한 전복[할리오티스(Haliotis)종]',
       '해파리(염장 또는 염수장한 것)', '미역(건조한 것)(식용의 것)', '톳(냉동한 것)(식용의 것)',
       '다랭이(기름담금)', '버니토우(기타)', '고등어(조제/밀폐용기에 넣은 것)', '꽁치(통조림외조제품)',
       '생선묵(게맛)', '어류조제,저장제품(생선 페이스트,마리네이드,소시지,생선묵)',
       '조미오징어(조제 또는 저장처리)(밀폐용기에 넣은 것)', '오징어 기타(조제 또는 저장처리)',
       '골뱅이(조제 또는 저장처리)(밀폐용기에 넣은 것)', '김', '조제한 해조류 김 제외', '천일염', '식염',
       '기타소금,해수', '가자미(플루로넥테스 플라테사)', '기타넙치류(냉동)',
       '꽁치[콜로라비스 사이라(Cololabis saira)]', '대구(가두스 모르화,가두스 오각,가두스 마크로세팔루스)',
       '아귀(냉동)', '민어(냉동)', '기타어류(냉동)', '명태(냉동한 피레트)(테라그라 찰코그라마)',
       '기타어류의 기타(냉동한 피레트)', '기타 냉동연육 외 기타',
       '틸라피아

In [46]:
df['수산물명'].nunique()

492

In [51]:
df['수산물명'].value_counts().sort_index().to_excel('MPRC_List.xlsx')

In [52]:
df['수산물명'].head()

0                                  기타 포유동물
1                                   넙치(활어)
2                    터벗(신선 또는 냉장)(프세타 맥시마)
3                   청어(클루페아 하렌구스,클루페아 팔라시)
4    고등어(스콤버 스콤브루스,스콤버 오스트랄라시쿠스,스콤버 자포니쿠스)
Name: 수산물명, dtype: object

In [59]:
# str accessor(접근자): 시리즈 원소에 대해 문자열을 처리하는 특별한 객체
# 문자열을 처리하는 다양한 함수를 가지고 있음(모듈)
# raw-string은 문자열 안에 있는 모든 이스케이프의 기능을 해제합니다.
df['수산물명'] = df['수산물명'].str.replace(pat = r'(\(|\[|\{).+', repl = '', regex = True)

In [60]:
df['수산물명'].value_counts().sort_index().to_excel('MPRC_List_2.xlsx')

In [61]:
cond = (df['국가명'] == '일본')
cond

0        False
1        False
2        False
3        False
4        False
         ...  
32375    False
32376    False
32377    False
32378    False
32379    False
Name: 국가명, Length: 32380, dtype: bool

In [62]:
cond.sum()

np.int64(2630)

In [63]:
# 불리언 인덱싱: 조건에 맞는 행 또는 열만 선택하기
jpn = df.loc[cond, :]

In [64]:
jpn.shape

(2630, 6)

In [65]:
jpn.head()

,기준년월,국가명,수산물명,수출입명,중량,달러
466,202401,일본,게살,수입,4.78,291
471,202401,일본,붉은 대게살,수출,171281.40,1942836
486,202401,일본,게 조제품,수출,20453.50,95247
487,202401,일본,게 조제품,수입,0.90,62
491,202401,일본,고등어,수출,14861.60,96138


In [67]:
jpn['달러'].sum()

np.int64(876403715)

In [68]:
jpn.loc[jpn['수출입명'] == '수출', '달러'].sum()

np.int64(667886640)

In [69]:
jpn.loc[jpn['수출입명'] == '수입', '달러'].sum()

np.int64(208517075)

In [70]:
jpn.groupby(by = '수출입명')['달러'].sum()

수출입명
수입    208517075
수출    667886640
Name: 달러, dtype: int64

In [72]:
jpn.groupby(by = '수산물명')['달러'].sum().sort_values()

수산물명
고등어의것                                       8
암염                                         62
기타 육·설육                                    76
틸라피아,메기,잉어,뱀장어,나일퍼치 종의 염장.염수장한 것           90
상어지느러미, 기타조제저장처리상어지느러미                     95
                                      ...    
돔                                    36746910
전복                                   38059950
방어                                   77102331
다랑어 기타                               92801196
김                                   206891664
Name: 달러, Length: 181, dtype: int64

In [73]:
jpn.loc[jpn['수출입명'] == '수출', :].groupby(by = '수산물명')['달러'].sum().sort_values()

수산물명
산호                            0
연어                            3
양식진주                          7
고등어의것                         8
어류의 지방과 기름 및 그 분획물           21
                        ...    
미역                     25107759
붕장어                    32329825
전복                     38059950
다랑어 기타                 92064577
김                     206325896
Name: 달러, Length: 144, dtype: int64

In [74]:
jpn.loc[jpn['수출입명'] == '수입', :].groupby(by = '수산물명')['달러'].sum().sort_values()

수산물명
건조한 것                3
기타어류의 피레트           19
바지락                 21
명란                  23
명태 냉동연육             25
                ...   
가리비과의조개        7557333
기타어류           8707267
가리비과의조개 기타    30841036
돔             36711223
방어            76845874
Name: 달러, Length: 138, dtype: int64

In [75]:
jpn.groupby(by = ['수출입명', '수산물명'])['달러'].sum().sort_values()

수출입명  수산물명  
수출    산호                0
      연어                3
수입    건조한 것             3
수출    양식진주              7
      고등어의것             8
                  ...    
수입    돔          36711223
수출    전복         38059950
수입    방어         76845874
수출    다랑어 기타     92064577
      김         206325896
Name: 달러, Length: 282, dtype: int64

In [78]:
pvt = pd.pivot_table(data = jpn, index = '수산물명', columns = '수출입명', 
                     values = '달러', aggfunc = 'sum', fill_value = 0)

In [80]:
pvt.head()

수출입명,수입,수출
수산물명,,
가다랑어 또는 줄무늬버니토우,3365,75058
가다랭이,13397,893886
가리비과의조개,7557333,9839
가리비과의조개 기타,30841036,0
가오리,2791,0


In [79]:
pvt.columns

Index(['수입', '수출'], dtype='object', name='수출입명')

In [81]:
pvt.index

Index(['가다랑어 또는 줄무늬버니토우', '가다랭이', '가리비과의조개', '가리비과의조개 기타', '가오리', '가오리와 홍어',
       '가자미', '간', '갈치', '갑각류 기타',
       ...
       '피조개 기타', '한천', '해삼', '해초류와 기타조류 기타', '해파리', '홍살치', '홍어', '홍합', '황다랑어',
       '황새치'],
      dtype='object', name='수산물명', length=181)

In [83]:
pvt = pvt.reset_index()

In [85]:
pvt.columns.name = None

In [86]:
pvt.head()

,수산물명,수입,수출
0,가다랑어 또는 줄무늬버니토우,3365,75058
1,가다랭이,13397,893886
2,가리비과의조개,7557333,9839
3,가리비과의조개 기타,30841036,0
4,가오리,2791,0


In [88]:
pvt['무역수지'] = pvt['수출'] - pvt['수입']

In [89]:
pvt.head()

,수산물명,수입,수출,무역수지
0,가다랑어 또는 줄무늬버니토우,3365,75058,71693
1,가다랭이,13397,893886,880489
2,가리비과의조개,7557333,9839,-7547494
3,가리비과의조개 기타,30841036,0,-30841036
4,가오리,2791,0,-2791


In [91]:
pvt.sort_values(by = '무역수지', ascending = False)

,수산물명,수입,수출,무역수지
47,김,565768,206325896,205760128
61,다랑어 기타,736619,92064577,91327958
146,전복,0,38059950,38059950
93,붕장어,0,32329825,32329825
80,미역,67548,25107759,25040211
...,...,...,...,...
40,기타어류,8707267,3925905,-4781362
2,가리비과의조개,7557333,9839,-7547494
3,가리비과의조개 기타,30841036,0,-30841036
69,돔,36711223,35687,-36675536


In [92]:
def mprcItemIndex(nation):
    cond = (df['국가명'] == nation)
    nat = df.loc[cond, :]
    pvt = pd.pivot_table(data = nat, index = '수산물명', columns = '수출입명', 
                         values = '달러', aggfunc = 'sum', fill_value = 0)
    pvt = pvt.reset_index()
    pvt.columns.name = None
    pvt['무역수지'] = pvt['수출'] - pvt['수입']
    pvt = pvt.sort_values(by = '무역수지', ascending = False)
    return pvt

In [93]:
mprcItemIndex(nation = '일본')

,수산물명,수입,수출,무역수지
47,김,565768,206325896,205760128
61,다랑어 기타,736619,92064577,91327958
146,전복,0,38059950,38059950
93,붕장어,0,32329825,32329825
80,미역,67548,25107759,25040211
...,...,...,...,...
40,기타어류,8707267,3925905,-4781362
2,가리비과의조개,7557333,9839,-7547494
3,가리비과의조개 기타,30841036,0,-30841036
69,돔,36711223,35687,-36675536


In [ ]:
# 인덱스에 '국가명', 컬럼스에 '수출입명'으로 설정하고 '달러' 합계를 피벗 테이블로 생성
# '수출'에서 '수입'을 뺀 '무역수지'를 추가하고 내림차순 정렬
